In [ ]:
from scipy.io import loadmat
import pandas as pd

file_paths = [
    '/content/drive/MyDrive/p300/S1.mat',
    '/content/drive/MyDrive/p300/S2.mat',
    '/content/drive/MyDrive/p300/S3.mat',
    '/content/drive/MyDrive/p300/S4.mat',
    '/content/drive/MyDrive/p300/S5.mat'
]

dfs = []

data = loadmat(file_paths[0])
eeg_data = data['y']
trigger_data = data['trig']

df1 = pd.DataFrame(eeg_data, columns=[f"Ch_{i+1}" for i in range(eeg_data.shape[1])])
df1['trigger'] = trigger_data
df1['subject'] = f"S1"

In [ ]:
df1.describe()

In [ ]:
df1.head()

In [ ]:
from scipy.signal import resample

def downsample_eeg_with_subject(data, original_fs, target_fs):
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    non_numeric_cols = ['subject']
    
    num_samples = int(len(data) * target_fs / original_fs)
    
    downsampled_numeric_data = pd.DataFrame()
    for col in numeric_cols:
        downsampled_numeric_data[col] = resample(data[col], num_samples)
    
    downsampled_non_numeric_data = pd.DataFrame()
    for col in non_numeric_cols:
        downsampled_non_numeric_data[col] = [data[col].iloc[0]] * num_samples
        
    downsampled_data = pd.concat([downsampled_numeric_data, downsampled_non_numeric_data], axis=1)
    
    return downsampled_data

original_fs = 250

downsampled_64Hz = downsample_eeg_with_subject(df1, original_fs, 64)
print("Downsampled to 64 Hz:", downsampled_64Hz.head())

downsampled_128Hz = downsample_eeg_with_subject(df1, original_fs, 128)
print("Downsampled to 128 Hz:", downsampled_128Hz.head())


In [ ]:
downsampled_64Hz.describe()

In [ ]:
downsampled_64Hz.head()

In [ ]:
from scipy.signal import resample
from scipy.fft import fft, fftfreq

# Function to downsample EEG data and extract Fourier amplitudes
def downsample_eeg_with_subject(data, original_fs, target_fs):
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    non_numeric_cols = ['subject'] if 'subject' in data.columns else []
    
    num_samples = int(len(data) * target_fs / original_fs)
    
    downsampled_numeric_data = pd.DataFrame()
    fourier_amplitudes = pd.DataFrame()
    
    for col in numeric_cols:
        # Resample the data (this uses Fourier internally)
        resampled_data = resample(data[col], num_samples)
        downsampled_numeric_data[col] = resampled_data
        
        # Compute FFT on the downsampled data
        fft_values = fft(resampled_data)
        amplitudes = np.abs(fft_values) / len(fft_values)
        
        # Take only the first half of the FFT results (positive frequencies)
        fourier_amplitudes[col] = amplitudes[:len(fft_values) // 2]
    
    # Frequency bins based on the target sampling rate
    freqs = fftfreq(num_samples, d=1/target_fs)[:num_samples // 2]
    fourier_amplitudes['Frequency (Hz)'] = freqs
    
    # Handle non-numeric columns
    downsampled_non_numeric_data = pd.DataFrame()
    for col in non_numeric_cols:
        downsampled_non_numeric_data[col] = [data[col].iloc[0]] * num_samples
        
    downsampled_data = pd.concat([downsampled_numeric_data, downsampled_non_numeric_data], axis=1)
    
    return downsampled_data, fourier_amplitudes

# Original sampling rate
original_fs = 250

# Downsample to 64 Hz
downsampled_64Hz, fourier_64Hz = downsample_eeg_with_subject(df1, original_fs, 64)
print("Downsampled to 64 Hz:", downsampled_64Hz.head())
print("Fourier Amplitudes for 64 Hz Data:", fourier_64Hz.head())

# Downsample to 128 Hz
downsampled_128Hz, fourier_128Hz = downsample_eeg_with_subject(df1, original_fs, 128)
print("Downsampled to 128 Hz:", downsampled_128Hz.head())
print("Fourier Amplitudes for 128 Hz Data:", fourier_128Hz.head())
